<a href="https://colab.research.google.com/github/anuraglahon16/AI-Engineering/blob/main/Evaluation_of_RAG_using_Ragas_Assignment_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Evaluation of RAG Using Ragas

In the following notebook we'll explore how to evaluate RAG pipelines using a powerful open-source tool called "Ragas". This will give us tools to evaluate component-wise metrics, as well as end-to-end metrics about the performance of our RAG pipelines.

In the following notebook we'll complete the following tasks:

- 🤝 Breakout Room #1:
  1. Install required libraries
  2. Set Environment Variables
  3. Creating a simple RAG pipeline with [LangChain v0.1.0](https://blog.langchain.dev/langchain-v0-1-0/)
  

- 🤝 Breakout Room #2:
  1. Synthetic Dataset Generation for Evaluation using the [Ragas](https://github.com/explodinggradients/ragas)
  2. Evaluating our pipeline with Ragas
  3. Making Adjustments to our RAG Pipeline
  4. Evaluating our Adjusted pipeline against our baseline
  5. Testing OpenAI's Claim

The only way to get started is to get started - so let's grab our dependencies for the day!

## Motivation

A claim, made by OpenAI, is that their `text-embedding-3-small` is better (generally) than their `text-embedding-ada-002` model.

Here's some passages from their [blog](https://openai.com/blog/new-embedding-models-and-api-updates) about the `text-embedding-3` release:

> `text-embedding-3-small` is our new highly efficient embedding model and provides a significant upgrade over its predecessor, the `text-embedding-ada-002` model...

> **Stronger performance.** Comparing `text-embedding-ada-002` to `text-embedding-3-small`, the average score on a commonly used benchmark for multi-language retrieval ([MIRACL](https://github.com/project-miracl/miracl)) has increased from 31.4% to 44.0%, while the average score on a commonly used benchmark for English tasks ([MTEB](https://github.com/embeddings-benchmark/mteb)) has increased from 61.0% to 62.3%.

Well, with a library like Ragas - we can put that claim to the test!

If what they claim is true - we should see an increase on related metrics by using the new embedding model!

# 🤝 Breakout Room #1

## Task 1: Installing Required Libraries

A reminder that one of the [key features](https://blog.langchain.dev/langchain-v0-1-0/) of LangChain v0.1.0 is the compartmentalization of the various LangChain ecosystem packages!

So let's begin grabbing all of our LangChain related packages!

In [3]:
!pip install -U -q langchain langchain-openai langchain_core langchain-community langchainhub openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 807.5/807.5 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 256.9/256.9 kB 22.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 54.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.4/227.4 kB 21.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.6/66.6 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 48.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.5/138.5 kB 17.3 MB/s eta 0:00:00


We'll also get the "star of the show" today, which is Ragas!

In [4]:
!pip install -qU ragas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.1/71.1 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.8 MB/s eta 0:00:00


As well, instead of the remote hosted solution that we used last week (Pinecone), we'll be leveraging Meta's [FAISS](https://github.com/facebookresearch/faiss) as the backend for our LangChain `VectorStore`.

We'll also install `unstructured` (from [Unstructured-IO](https://github.com/Unstructured-IO/unstructured)) and its dependencies which will allow us to load PDFs using the `UnstructuredPDFLoader` in the `langchain-community` package!

In [5]:
!pip install -qU faiss_cpu pymupdf pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 37.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 36.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 48.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.6/30.6 MB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.4/345.4 kB 27.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 0.22.0 requires pandas<2.1.4,>=1.5.0, but you have pandas 2.2.1 which is incompatible.
google-colab 1.0.0 requires pandas==1.5.3, but you have pandas 2.2.1 which is incompatible.


## Task 2: Set Environment Variables

Let's set up our OpenAI API key so we can leverage their API later on.

In [6]:
import os
import openai
from getpass import getpass

openai.api_key = getpass("Please provide your OpenAI Key: ")
os.environ["OPENAI_API_KEY"] = openai.api_key

Please provide your OpenAI Key: ··········


## Task 3: Creating a Simple RAG Pipeline with LangChain v0.1.0

Building on what we learned last week, we'll be leveraging LangChain v0.1.0 and LCEL to build a simple RAG pipeline that we can baseline with Ragas.

## Building our RAG pipeline

Let's review the basic steps of RAG again:

- Create an Index
- Use retrieval to obtain pieces of context from our Index that are similar to our query
- Use a LLM to generate responses based on the retrieved context

Let's get started by creating our index.

> NOTE: We're going to start leaning on the term "index" to refer to our `VectorStore`, `VectorDatabase`, etc. We can think of "index" as the catch-all term, whereas `VectorStore` and the like relate to the specific technologies used to create, store, and interact with the index.

### Creating an Index

You'll notice that the largest changes (outside of some import changes) are that our old favourite chains are back to being bundled in an easily usable abstraction.

We can still create custom chains using LCEL - but we can also be more confident that our pre-packaged chains are creating using LCEL under the hood.

#### Loading Data

Let's start by loading some data!

> NOTE: You'll notice that we're using a document loader from the community package of LangChain. This is part of the v0.1.0 changes that make the base (`langchain-core`) package remain lightweight while still providing access to some of the more powerful community integrations.

In [7]:
!git clone https://github.com/AI-Maker-Space/DataRepository

Cloning into 'DataRepository'...
remote: Enumerating objects: 54, done.
remote: Counting objects: 100% (46/46), done.
remote: Compressing objects: 100% (34/34), done.
remote: Total 54 (delta 15), reused 20 (delta 7), pack-reused 8
Receiving objects: 100% (54/54), 51.28 MiB | 30.98 MiB/s, done.
Resolving deltas: 100% (15/15), done.


In [8]:
from langchain_community.document_loaders import PyMuPDFLoader

loader = PyMuPDFLoader(
    "DataRepository/MuskComplaint.pdf",
)

documents = loader.load()

In [9]:
documents[0].metadata

{'source': 'DataRepository/MuskComplaint.pdf',
 'file_path': 'DataRepository/MuskComplaint.pdf',
 'page': 0,
 'total_pages': 46,
 'format': 'PDF 1.7',
 'title': '',
 'author': '',
 'subject': '',
 'keywords': '',
 'creator': '',
 'producer': '',
 'creationDate': '',
 'modDate': '',
 'trapped': ''}

#### Transforming Data

Now that we've got our single document - let's split it into smaller pieces so we can more effectively leverage it with our retrieval chain!

We'll start with the classic: `RecursiveCharacterTextSplitter`.

In [10]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 700,
    chunk_overlap = 50
)

documents = text_splitter.split_documents(documents)

Let's confirm we've split our document.

In [11]:
len(documents)

159

#### Loading OpenAI Embeddings Model

We'll need a process by which we can convert our text into vectors that allow us to compare to our query vector.

Let's use OpenAI's `text-embedding-ada-002` for this task!

In [12]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(
    model="text-embedding-ada-002"
)

#### Creating a FAISS VectorStore

Now that we have documents - we'll need a place to store them alongside their embeddings.

In [13]:
from langchain_community.vectorstores import FAISS

vector_store = FAISS.from_documents(documents, embeddings)

####❓ Question #1:

List out a few of the techniques that FAISS uses that make it performant.

1. Quantization : FAISS employs vector quantization, like Product Quantization, to compress vectors, minimizing memory use and accelerating searches by approximating distances on compressed vectors, enabling efficient handling of billions of vectors.
2.Indexing Structures: FAISS employs advanced indexing structures, such as Hierarchical Navigable Small World (HNSW) and Navigable Small World (NSG) graphs, to accelerate the search process

> NOTE: Check the [repository](https://github.com/facebookresearch/faiss) for more information about FAISS!

#### Creating a Retriever

To complete our index, all that's left to do is expose our vectorstore as a retriever - which we can do the same way we would in previous version of LangChain!

In [14]:
retriever = vector_store.as_retriever()

#### Testing our Retriever

Now that we've gone through the trouble of creating our retriever - let's see it in action!

In [15]:
retrieved_documents = retriever.invoke("Who is the plantiff?")

In [16]:
for doc in retrieved_documents:
  print(doc)

page_content='would be owned by the foundation and used ‘for the good of the world’[.]” Plaintiff \nreplied: “Agree on all.” Ex. 2 at 1.' metadata={'source': 'DataRepository/MuskComplaint.pdf', 'file_path': 'DataRepository/MuskComplaint.pdf', 'page': 27, 'total_pages': 46, 'format': 'PDF 1.7', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'creator': '', 'producer': '', 'creationDate': '', 'modDate': '', 'trapped': ''}
page_content='property and derivative works funded by those monies, Plaintiff is presently unable to ascertain his \ninterest in or the use, allocation, or distribution of assets without an accounting. Plaintiff is therefore \nentitled to an accounting.' metadata={'source': 'DataRepository/MuskComplaint.pdf', 'file_path': 'DataRepository/MuskComplaint.pdf', 'page': 32, 'total_pages': 46, 'format': 'PDF 1.7', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'creator': '', 'producer': '', 'creationDate': '', 'modDate': '', 'trapped': ''}
page_content='1

### Creating a RAG Chain

Now that we have the "R" in RAG taken care of - let's look at creating the "AG"!

#### Creating a Prompt Template

There are a few different ways we could create our prompt template - we could create a custom template, as seen in the code below, or we could simply pull a prompt from the prompt hub! Let's look at an example of that!

In [17]:
from langchain import hub

retrieval_qa_prompt = hub.pull("langchain-ai/retrieval-qa-chat")

In [18]:
print(retrieval_qa_prompt.messages[0].prompt.template)

Answer any use questions based solely on the context below:

<context>
{context}
</context>


As you can see - the prompt template is simple (and has a small error) - so we'll create our own to be a bit more specific!

In [19]:
from langchain.prompts import ChatPromptTemplate

template = """Answer the question based only on the following context. If you cannot answer the question with the context, please respond with 'I don't know':

Context:
{context}

Question:
{question}
"""

prompt = ChatPromptTemplate.from_template(template)

#### Setting Up our Basic QA Chain

Now we can instantiate our basic RAG chain!

We'll use LCEL directly just to see an example of it - but you could just as easily use an abstraction here to achieve the same goal!

We'll also ensure to pass-through our context - which is critical for RAGAS.

In [20]:
from operator import itemgetter

from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

primary_qa_llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

retrieval_augmented_qa_chain = (
    # INVOKE CHAIN WITH: {"question" : "<<SOME USER QUESTION>>"}
    # "question" : populated by getting the value of the "question" key
    # "context"  : populated by getting the value of the "question" key and chaining it into the base_retriever
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    # "context"  : is assigned to a RunnablePassthrough object (will not be called or considered in the next step)
    #              by getting the value of the "context" key from the previous step
    | RunnablePassthrough.assign(context=itemgetter("context"))
    # "response" : the "context" and "question" values are used to format our prompt object and then piped
    #              into the LLM and stored in a key called "response"
    # "context"  : populated by getting the value of the "context" key from the previous step
    | {"response": prompt | primary_qa_llm, "context": itemgetter("context")}
)

####🏗️ Activity #1:

Describe the pipeline shown above in simple terms. You can include a diagram if desired.

The pipeline takes a user's question, uses it to gather relevant information via a retriever, and then feeds both the question and the retrieved context into a GPT-3.5-turbo model to formulate an answer. It sequentially extracts the question, obtains related context, and combines these inputs to prompt the language model for a response.

Let's test it out!

In [21]:
question = "Who is the plantiff?"

result = retrieval_augmented_qa_chain.invoke({"question" : question})

print(result["response"].content)

Elon Musk


In [22]:
question = "What does this complaint pertain to?"

result = retrieval_augmented_qa_chain.invoke({"question" : question})

print(result["response"].content)
print(result["context"])

The complaint pertains to breach of fiduciary duty, unfair business practices, and accounting.
[Document(page_content='1 \n2 \n3 \n4 \n5 \n6 \n7 \n8 \n9 \n10 \n11 \n12 \n13 \n14 \n15 \n16 \n17 \n18 \n19 \n20 \n21 \n22 \n23 \n24 \n25 \n26 \n27 \n28 \n \n \n– 31 – \nCOMPLAINT \n \nTHIRD CAUSE OF ACTION \nBreach of Fiduciary Duty  \nAgainst All Defendants \n133. \nPlaintiff realleges and incorporates by reference only paragraphs of this Complaint \nnecessary for his claim of Breach of Fiduciary Duty. \n134. \nUnder California law, Defendants owe fiduciary duties to Plaintiff, including a duty \nto use Plaintiff’s contributions for the purposes for which they were made. E.g., Cal. Bus. & Prof. \nCode § 17510.8. Defendants have repeatedly breached their fiduciary duties to Plaintiff, including \nby:', metadata={'source': 'DataRepository/MuskComplaint.pdf', 'file_path': 'DataRepository/MuskComplaint.pdf', 'page': 30, 'total_pages': 46, 'format': 'PDF 1.7', 'title': '', 'author': '', 'subject

We can already see that there are some improvements we could make here.

For now, let's switch gears to RAGAS to see how we can leverage that tool to provide us insight into how our pipeline is performing!

# 🤝 Breakout Room #2

## Task 1: Synthetic Dataset Generation for Evaluation using Ragas

Ragas is a powerful library that lets us evaluate our RAG pipeline by collecting input/output/context triplets and obtaining metrics relating to a number of different aspects of our RAG pipeline.

We'll be evluating on every core metric today, but in order to do that - we'll need to creat a test set. Luckily for us, Ragas can do that directly!

### Synthetic Test Set Generation

We can leverage Ragas' [`Synthetic Test Data generation`](https://docs.ragas.io/en/stable/concepts/testset_generation.html) functionality to generate our own synthetic QC pairs - as well as a synthetic ground truth - quite easily!

> NOTE: This process will use `gpt-3.5-turbo-16k` as the base generator and `gpt-4` as the critic - if you're attempting to create a lot of samples please be aware of cost, as well as rate limits.

In [23]:
eval_documents = documents

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1500,
    chunk_overlap = 400
)

eval_documents = text_splitter.split_documents(eval_documents)

####❓ Question #2:

Why is it important to split our documents using different parameters when creating our synthetic data?


Splitting documents into chunks using different parameters for synthetic data creation is important because it allows for tailored processing of text, ensuring efficient model handling and preserving context integrity. This approach adapts to the model's limitations and optimizes data representation, crucial for generating high-quality, contextually accurate synthetic datasets.

In [24]:
len(documents)

159

In [25]:
from ragas.testset.generator import TestsetGenerator
from ragas.testset.evolutions import simple, reasoning, multi_context

generator = TestsetGenerator.with_openai()

testset = generator.generate_with_langchain_docs(documents, test_size=10, distributions={simple: 0.25, reasoning: 0.25, multi_context: 0.5})

embedding nodes:   0%|          | 0/318 [00:00<?, ?it/s]

Generating:   0%|          | 0/10 [00:00<?, ?it/s]

####❓ Question #3:

`{simple: 0.5, reasoning: 0.25, multi_context: 0.25}`

What exactly does this mapping refer to?

50% of the questions are intended to be simple, 25% based on reasoning, and 25% on multi-context understanding.

> NOTE: Check out the Ragas documentation on this generation process [here](https://docs.ragas.io/en/stable/concepts/testset_generation.html).

Let's look at the output and see what we can learn about it!

In [26]:
testset.test_data[0]

DataRow(question='What is the role of IP assets in the proposed business model?', contexts=['business model were valid, it would radically redefine how venture capitalism is practiced in \nCalifornia and beyond. Rather than start out as a for-profit entity from the outset, “smart” investors \nwould establish non-profits, use pre-tax donations to fund research and development, and then once \ntheir technology had been developed and proven, would slide the resulting IP assets into a new for-\nprofit venture to enrich themselves and their profit-maximizing corporate partners. That is not \nsupposed to be how the law works in California or anywhere else in this country, and this should \nnot be the first Court to hold otherwise. \n114.'], ground_truth='The role of IP assets in the proposed business model is to be slid into a new for-profit venture to enrich the investors and their corporate partners.', evolution_type='simple')

### Generating Responses with RAG Pipeline

Now that we have some QC pairs, and some ground truths, let's evaluate our RAG pipeline using Ragas.

The process is, again, quite straightforward - thanks to Ragas and LangChain!

Let's start by extracting our questions and ground truths from our create testset.

We can start by converting our test dataset into a Pandas DataFrame.

In [27]:
test_df = testset.to_pandas()

In [28]:
test_df

,question,contexts,ground_truth,evolution_type,episode_done
0,What is the role of IP assets in the proposed ...,"[business model were valid, it would radically...",The role of IP assets in the proposed business...,simple,True
1,How did OpenAI use reinforcement learning in t...,[77. \nInitial work at OpenAI followed much in...,OpenAI used reinforcement learning to play Dot...,simple,True
2,"""Why are there more parameters in OpenAI's GPT...",[1 \n2 \n3 \n4 \n5 \n6 \n7 \n8 \n9 \n10 \n11 \...,nan,reasoning,True
3,How did researchers demonstrate GPT-3's comple...,[implementation for others to build on. \n84. ...,Researchers demonstrated GPT-3's complex reaso...,reasoning,True
4,What are the benefits of releasing open versio...,"[challenging.” At the time, OpenAI stated that...",The benefits of releasing open versions of pow...,multi_context,True
5,"""What game did OpenAI use reinforcement learni...",[77. \nInitial work at OpenAI followed much in...,OpenAI used reinforcement learning in the game...,multi_context,True
6,"How did Alphabet, Inc. respond to concerns abo...","[Page, then-CEO of Google’s parent company Alp...",nan,multi_context,True
7,What concerns arise from investments in OpenAI...,[chip fabrication plants. If $10 billion from ...,"The concerns arise from investments in OpenAI,...",multi_context,True
8,What were Stephen Hawking's concerns about AGI...,[18. \nMr. Musk has long recognized that AGI p...,Stephen Hawking's concerns about AGI in the wr...,multi_context,True
9,What architecture and pre-training method did ...,[those connections to the target language. \n7...,OpenAI used the first half of Google's Transfo...,reasoning,True


In [29]:
test_questions = test_df["question"].values.tolist()
test_groundtruths = test_df["ground_truth"].values.tolist()

Now we'll generate responses using our RAG pipeline using the questions we've generated - we'll also need to collect our retrieved contexts for each question.

We'll do this in a simple loop to see exactly what's happening!

In [30]:
answers = []
contexts = []

for question in test_questions:
  response = retrieval_augmented_qa_chain.invoke({"question" : question})
  answers.append(response["response"].content)
  contexts.append([context.page_content for context in response["context"]])

Now we can wrap our information in a Hugging Face dataset for use in the Ragas library.

In [31]:
from datasets import Dataset

response_dataset = Dataset.from_dict({
    "question" : test_questions,
    "answer" : answers,
    "contexts" : contexts,
    "ground_truth" : test_groundtruths
})

Let's take a peek and see what that looks like!

In [32]:
response_dataset[0]

{'question': 'What is the role of IP assets in the proposed business model?',
 'answer': 'The role of IP assets in the proposed business model is to be developed and proven through pre-tax donations, then transferred to a new for-profit venture to enrich investors and corporate partners.',
 'contexts': ['business model were valid, it would radically redefine how venture capitalism is practiced in \nCalifornia and beyond. Rather than start out as a for-profit entity from the outset, “smart” investors \nwould establish non-profits, use pre-tax donations to fund research and development, and then once \ntheir technology had been developed and proven, would slide the resulting IP assets into a new for-\nprofit venture to enrich themselves and their profit-maximizing corporate partners. That is not \nsupposed to be how the law works in California or anywhere else in this country, and this should \nnot be the first Court to hold otherwise. \n114.',
  'and, ultimately, the public. From an inv

## Task 2: Evaluating our Pipeline with Ragas

Now that we have our response dataset - we can finally get into the "meat" of Ragas - evaluation!

First, we'll import the desired metrics, then we can use them to evaluate our created dataset!

Check out the specific metrics we'll be using in the Ragas documentation:

- [Faithfulness](https://docs.ragas.io/en/stable/concepts/metrics/faithfulness.html)
- [Answer Relevancy](https://docs.ragas.io/en/stable/concepts/metrics/answer_relevance.html)
- [Context Precision](https://docs.ragas.io/en/stable/concepts/metrics/context_precision.html)
- [Context Recall](https://docs.ragas.io/en/stable/concepts/metrics/context_recall.html)
- [Answer Correctness](https://docs.ragas.io/en/stable/concepts/metrics/answer_correctness.html)

See the accompanied presentation for more in-depth explanations about each of the metrics!

In [33]:
from ragas import evaluate
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    answer_correctness,
    context_recall,
    context_precision,
)

metrics = [
    faithfulness,
    answer_relevancy,
    context_recall,
    context_precision,
    answer_correctness,
]

All that's left to do is call "evaluate" and away we go!

In [34]:
results = evaluate(response_dataset, metrics)

Evaluating:   0%|          | 0/50 [00:00<?, ?it/s]

In [35]:
results

{'faithfulness': 0.9750, 'answer_relevancy': 0.9496, 'context_recall': 0.7500, 'context_precision': 0.7750, 'answer_correctness': 0.6223}

In [36]:
results_df = results.to_pandas()
results_df

,question,answer,contexts,ground_truth,faithfulness,answer_relevancy,context_recall,context_precision,answer_correctness
0,What is the role of IP assets in the proposed ...,The role of IP assets in the proposed business...,"[business model were valid, it would radically...",The role of IP assets in the proposed business...,1.00,0.999999,1.0,1.00,0.738892
1,How did OpenAI use reinforcement learning in t...,OpenAI used reinforcement learning to compete ...,[77. \nInitial work at OpenAI followed much in...,OpenAI used reinforcement learning to play Dot...,1.00,0.971244,1.0,1.00,0.746960
2,"""Why are there more parameters in OpenAI's GPT...",The GPT-3 model has more parameters to make it...,[1 \n2 \n3 \n4 \n5 \n6 \n7 \n8 \n9 \n10 \n11 \...,nan,1.00,0.877751,0.0,0.00,0.186036
3,How did researchers demonstrate GPT-3's comple...,Researchers demonstrated GPT-3's complex reaso...,[implementation for others to build on. \n84. ...,Researchers demonstrated GPT-3's complex reaso...,1.00,0.958730,1.0,1.00,0.738175
4,What are the benefits of releasing open versio...,The benefits of releasing open versions of pow...,"[challenging.” At the time, OpenAI stated that...",The benefits of releasing open versions of pow...,1.00,0.970252,1.0,1.00,0.746396
5,"""What game did OpenAI use reinforcement learni...",OpenAI used reinforcement learning to compete ...,[77. \nInitial work at OpenAI followed much in...,OpenAI used reinforcement learning in the game...,1.00,0.930770,0.5,1.00,0.539319
6,"How did Alphabet, Inc. respond to concerns abo...","Alphabet, Inc. responded by establishing a boa...",[1 \n2 \n3 \n4 \n5 \n6 \n7 \n8 \n9 \n10 \n11 \...,nan,0.75,0.942701,0.0,0.00,0.182998
7,What concerns arise from investments in OpenAI...,The concerns arise from potential investments ...,[chip fabrication plants. If $10 billion from ...,"The concerns arise from investments in OpenAI,...",1.00,0.941258,1.0,1.00,0.748291
8,What were Stephen Hawking's concerns about AGI...,Stephen Hawking's concerns about AGI in the wr...,[18. \nMr. Musk has long recognized that AGI p...,Stephen Hawking's concerns about AGI in the wr...,1.00,0.964491,1.0,1.00,0.600600
9,What architecture and pre-training method did ...,OpenAI used the first half of Google's Transfo...,[those connections to the target language. \n7...,OpenAI used the first half of Google's Transfo...,1.00,0.939269,1.0,0.75,0.995719


## Task 3: Making Adjustments to our RAG Pipeline

Now that we have established a baseline - we can see how any changes impact our pipeline's performance!

Let's modify our retriever and see how that impacts our Ragas metrics!

> NOTE: MultiQueryRetriever is expanded on [here](https://python.langchain.com/docs/modules/data_connection/retrievers/MultiQueryRetriever) but for now, the implementation is not important to our lesson!

In [37]:
from langchain.retrievers import MultiQueryRetriever

advanced_retriever = MultiQueryRetriever.from_llm(retriever=retriever, llm=primary_qa_llm)

We'll also re-create our RAG pipeline using the abstractions that come packaged with LangChain v0.1.0!

First, let's create a chain to "stuff" our documents into our context!

In [38]:
from langchain.chains.combine_documents import create_stuff_documents_chain

document_chain = create_stuff_documents_chain(primary_qa_llm, retrieval_qa_prompt)

Next, we'll create the retrieval chain!

In [39]:
from langchain.chains import create_retrieval_chain

retrieval_chain = create_retrieval_chain(advanced_retriever, document_chain)

In [40]:
response = retrieval_chain.invoke({"input": "Who is the plantiff?"})

In [41]:
print(response["answer"])

The plaintiff is Elon Musk.


In [42]:
response = retrieval_chain.invoke({"input": "What does this complaint pertain to?"})

In [43]:
print(response["answer"])

The complaint pertains to a legal case involving Plaintiff Elon Musk alleging breach of fiduciary duty, unfair business practices under California Business and Professions Code sections 17200 et seq, and demanding an accounting, restitution, disgorgement of monies received by Defendants, prejudgment interest, and specific performance. The complaint also includes a demand for a jury trial.


Well, just from those responses this chain *feels* better - but lets see how it performs on our eval!

Let's do the same process we did before to collect our pipeline's contexts and answers.

In [44]:
answers = []
contexts = []

for question in test_questions:
  response = retrieval_chain.invoke({"input" : question})
  answers.append(response["answer"])
  contexts.append([context.page_content for context in response["context"]])

Now we can convert this into a dataset, just like we did before.

In [45]:
response_dataset_advanced_retrieval = Dataset.from_dict({
    "question" : test_questions,
    "answer" : answers,
    "contexts" : contexts,
    "ground_truth" : test_groundtruths
})

Let's evaluate on the same metrics we did for the first pipeline and see how it does!

In [46]:
advanced_retrieval_results = evaluate(response_dataset_advanced_retrieval, metrics)

Evaluating:   0%|          | 0/50 [00:00<?, ?it/s]

In [47]:
advanced_retrieval_results_df = advanced_retrieval_results.to_pandas()
advanced_retrieval_results_df

,question,answer,contexts,ground_truth,faithfulness,answer_relevancy,context_recall,context_precision,answer_correctness
0,What is the role of IP assets in the proposed ...,"In the proposed business model, investors woul...","[business model were valid, it would radically...",The role of IP assets in the proposed business...,1.0,0.885696,1.000000,1.00000,0.522232
1,How did OpenAI use reinforcement learning in t...,OpenAI used reinforcement learning to compete ...,[77. \nInitial work at OpenAI followed much in...,OpenAI used reinforcement learning to play Dot...,1.0,0.867406,1.000000,1.00000,0.617547
2,"""Why are there more parameters in OpenAI's GPT...",OpenAI increased the number of parameters in t...,[1 \n2 \n3 \n4 \n5 \n6 \n7 \n8 \n9 \n10 \n11 \...,nan,1.0,0.929528,0.000000,0.00000,0.190278
3,How did researchers demonstrate GPT-3's comple...,Researchers at the University of Tokyo and Goo...,[implementation for others to build on. \n84. ...,Researchers demonstrated GPT-3's complex reaso...,1.0,0.944104,1.000000,1.00000,0.734162
4,What are the benefits of releasing open versio...,The benefits of releasing open versions of pow...,"[challenging.” At the time, OpenAI stated that...",The benefits of releasing open versions of pow...,1.0,0.969340,1.000000,1.00000,0.808899
5,"""What game did OpenAI use reinforcement learni...",OpenAI used reinforcement learning in the stra...,[77. \nInitial work at OpenAI followed much in...,OpenAI used reinforcement learning in the game...,1.0,0.911771,0.666667,1.00000,0.535368
6,"How did Alphabet, Inc. respond to concerns abo...","Alphabet, Inc., the parent company of Google, ...","[2015, it appeared that Mr. Musk may have foun...",nan,NaN,1.000000,0.000000,0.00000,0.184529
7,What concerns arise from investments in OpenAI...,The concerns that arise from investments in Op...,[chip fabrication plants. If $10 billion from ...,"The concerns arise from investments in OpenAI,...",1.0,0.947612,1.000000,0.79619,0.926731
8,What were Stephen Hawking's concerns about AGI...,"Stephen Hawking, along with other luminaries l...",[18. \nMr. Musk has long recognized that AGI p...,Stephen Hawking's concerns about AGI in the wr...,1.0,0.924122,1.000000,1.00000,0.532241
9,What architecture and pre-training method did ...,OpenAI used the first half of Google's Transfo...,[those connections to the target language. \n7...,OpenAI used the first half of Google's Transfo...,1.0,0.947250,1.000000,1.00000,0.995719


## Task 4: Evaluating our Adjusted Pipeline Against Our Baseline

Now we can compare our results and see what directional changes occured!

Let's refresh with our initial metrics.

In [48]:
results

{'faithfulness': 0.9750, 'answer_relevancy': 0.9496, 'context_recall': 0.7500, 'context_precision': 0.7750, 'answer_correctness': 0.6223}

And see how our advanced retrieval modified our chain!

In [49]:
advanced_retrieval_results

{'faithfulness': 1.0000, 'answer_relevancy': 0.9327, 'context_recall': 0.7667, 'context_precision': 0.7796, 'answer_correctness': 0.6048}

In [50]:
import pandas as pd

df_original = pd.DataFrame(list(results.items()), columns=['Metric', 'Baseline'])
df_comparison = pd.DataFrame(list(advanced_retrieval_results.items()), columns=['Metric', 'MultiQueryRetriever with Document Stuffing'])

df_merged = pd.merge(df_original, df_comparison, on='Metric')

df_merged['Delta'] = df_merged['MultiQueryRetriever with Document Stuffing'] - df_merged['Baseline']

df_merged

,Metric,Baseline,MultiQueryRetriever with Document Stuffing,Delta
0,faithfulness,0.975000,1.000000,0.025000
1,answer_relevancy,0.949646,0.932683,-0.016964
2,context_recall,0.750000,0.766667,0.016667
3,context_precision,0.775000,0.779619,0.004619
4,answer_correctness,0.622339,0.604771,-0.017568


## Task 5: Testing OpenAI's Claim

Now that we've seen how our retriever can impact the performance of our RAG pipeline - let's see how changing our embedding model impacts performance.

####🏗️ Activity #2:

Please provide markdown, or code comments, to explain which each of the following steps are doing!

In [51]:
#Initializes a new embeddings model using OpenAI's "text-embedding-3-small". This model is used to convert text documents into vector embeddings.
new_embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

In [52]:
#Creates a FAISS vector store from the documents by converting each document into a vector using the initialized embeddings model.
vector_store = FAISS.from_documents(documents, new_embeddings)

In [53]:
# Converts the FAISS vector store into a retriever that can be used to find relevant documents based on query vectors.
new_retriever = vector_store.as_retriever()

In [54]:
#Enhances the basic retriever with the capability to use a language model for processing queries, enabling more sophisticated retrieval that can understand natural language queries better.
new_advanced_retriever = MultiQueryRetriever.from_llm(retriever=new_retriever, llm=primary_qa_llm)

In [55]:
#Creates a retrieval chain that uses the advanced retriever to fetch relevant documents based on the questions asked, integrating it into an existing document processing chain.
new_retrieval_chain = create_retrieval_chain(new_advanced_retriever, document_chain)

In [56]:
answers = []
contexts = []
#Iterates over a list of test questions, invoking the retrieval chain for each question to get a response,
#which includes an answer and the contexts it was based on.
for question in test_questions:
  response = new_retrieval_chain.invoke({"input" : question})
  answers.append(response["answer"])
  contexts.append([context.page_content for context in response["context"]])

In [57]:
#Compiles the answers and contexts retrieved for each question into a structured dataset for evaluation.

new_response_dataset_advanced_retrieval = Dataset.from_dict({
    "question" : test_questions,
    "answer" : answers,
    "contexts" : contexts,
    "ground_truth" : test_groundtruths
})

In [58]:
#Evaluates the performance of the retrieval chain using the compiled dataset and specified metrics, generating a results summary
new_advanced_retrieval_results = evaluate(new_response_dataset_advanced_retrieval, metrics)

Evaluating:   0%|          | 0/50 [00:00<?, ?it/s]

In [59]:
new_advanced_retrieval_results

{'faithfulness': 1.0000, 'answer_relevancy': 0.9206, 'context_recall': 0.7500, 'context_precision': 0.7617, 'answer_correctness': 0.5962}

In [60]:
#create pandas DataFrames to organize and compare the results from different retrieval configurations and baseline performances.
df_baseline = pd.DataFrame(list(results.items()), columns=['Metric', 'Baseline'])
df_original = pd.DataFrame(list(advanced_retrieval_results.items()), columns=['Metric', 'ADA'])
df_comparison = pd.DataFrame(list(new_advanced_retrieval_results.items()), columns=['Metric', 'Text Embedding 3'])

df_merged = pd.merge(df_original, df_comparison, on='Metric')
df_merged = pd.merge(df_baseline, df_merged, on="Metric")
#Calculates the performance difference (delta) between the new "text-embedding-3" based retriever and other configurations or baselines
df_merged['Delta - TE3 -> ADA'] = df_merged['Text Embedding 3'] - df_merged['ADA']
df_merged['Delta - TE3 -> Baseline'] = df_merged['Text Embedding 3'] - df_merged['Baseline']

df_merged

,Metric,Baseline,ADA,Text Embedding 3,Delta - TE3 -> ADA,Delta - TE3 -> Baseline
0,faithfulness,0.975000,1.000000,1.000000,0.000000,0.025000
1,answer_relevancy,0.949646,0.932683,0.920586,-0.012096,-0.029060
2,context_recall,0.750000,0.766667,0.750000,-0.016667,0.000000
3,context_precision,0.775000,0.779619,0.761667,-0.017952,-0.013333
4,answer_correctness,0.622339,0.604771,0.596160,-0.008610,-0.026178


####❓ Question #4:

Do you think, in your opinion, `text-embedding-3-small` is significantly better than `ada`?

`text-embedding-3-small` improves in context precision and maintains high faithfulness and context recall, suggesting advantages in certain areas over "ADA." However, its decrease in answer correctness raises concerns for tasks where accuracy is crucial. Whether it's significantly better depends on specific application priorities, balancing precision and relevancy against correctness.

## BONUS ACTIVITY: Showcase Multi-Context Perfomance Changes

Now that we've looked at a number of different examples - showcase the difference on the multi-context *specific* questions that were synthetically generated.

> NOTE: You have all the data you'll need already in the notebook if you made it to this step!

In [61]:
### YOUR CODE HERE